In [0]:

print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-aso/unified/app-tech.aso.sov-search-ads-keyword-fact.v1/fact/granularity=daily/date=2020-02-18/").select("keyword_id").distinct().count()

In [0]:



spark.read.format("delta").option("basePath", "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/").parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/par*").where("process_date between '2020-02-11' and '2020-02-14'").groupBy("_identifier").count().show(100)

In [0]:

from pyspark.sql.functions import desc
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review_load_temp/review_load_temp_v4/", depth_is_1=True)
print path_list[107]
for path in path_list[120:]:
    dup_list = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/{}/'.format(path.split("/")[3])).select('review_id','process_date','process_hour').distinct().groupBy("review_id").agg({"*":"count"}).where("count(1)>1").orderBy("count(1)",ascending = False).take(3)

    for x in dup_list:
        result1 = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/{}'.format(path.split("/")[3])).filter("review_id='{}'".format(x["review_id"])).groupBy("process_date","process_hour","time").agg({"*":"count"}).orderBy(desc("time"),desc("process_date"),desc("process_hour")).head(1)
        result2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/{}".format(path.split("/")[3])).filter("review_id='{}'".format(x["review_id"])).head(1)
        # print result1,result2
        if result1[0]["process_date"]==result2[0]["process_date"] and result1[0]["process_hour"]==result2[0]["process_hour"] and result1[0]["time"]==result2[0]["time"]  :
            print 'pass', path
        else:
            print 'failed!!!',result1, result2,x

In [0]:



spark.read.parquet('s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/event_month=2019-01/').filter("review_id='gp:AOqpTOHQ9qg91CJhYS7vTwxcJfBnLAr1dzOTDcruejCHxrsuI33EjSzLglUghYCivJ2lqj5H452DXKhrsNnEZw'").orderBy("process_date","process_hour",ascending=False).show()

In [0]:

spark.read.parquet('s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/event_month=2019-01/').filter("review_id='gp:AOqpTOHQ9qg91CJhYS7vTwxcJfBnLAr1dzOTDcruejCHxrsuI33EjSzLglUghYCivJ2lqj5H452DXKhrsNnEZw'").orderBy("process_date",ascending=False).show()

In [0]:



dup_list = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/event_month=2019-01/').select('review_id','process_date','process_hour','time').distinct().groupBy("review_id").agg({"*":"count"}).where("count(1)>1").orderBy("count(1)",ascending = False).show(50,False)

In [0]:


from pyspark.sql.functions import count, avg
spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/event_month=2019-01").select('review_id','time').groupBy("review_id","time").agg(count("time")).alias("count_time").groupBy("review_id","count_time.count(time)").withColumn('time_order', agg(count("count_time.count(time)"))).show(50,False)

In [0]:

import pyspark.sql.functions as F

review_df = spark.read.parquet(
    "s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/event_month=2018-01")
review_df.registerTempTable("reviews")
ios_samples_df = spark.sql("SELECT * FROM reviews WHERE market_code = 'apple-store' and user_review_url is not null LIMIT 10").cache()
ios_samples_df.count()
android_samples_df = spark.sql("SELECT * FROM reviews WHERE market_code = 'google-play' LIMIT 10").cache()
android_samples_df.count()
samples = []
for idx, df in [
    ("int-ss-review_v1-apple-store-ios-all-201701", ios_samples_df),
    ("int-ss-review_v1-google-play-android-all-201701", android_samples_df)]:
    print df.columns
    rows = (
        df
            .withColumn("date", F.date_format("time", "yyyy-MM-dd"))
            .drop("time")
            .withColumn("chinese_content",
                        F.when((df.content_language == "zh") | (df.content_language == "zh_Hant"), df.content))
            .withColumn("chinese_reply",
                        F.when((df.reply_language == "zh") | (df.reply_language == "zh_Hant"), df.reply))
            .withColumn("chinese_title",
                        F.when((df.title_language == "zh") | (df.title_language == "zh_Hant"), df.title))
            .withColumn("english_content", F.when(df.content_language == "en", df.content))
            .withColumn("english_reply", F.when(df.reply_language == "en", df.reply))
            .withColumn("english_title", F.when(df.title_language == "en", df.title))
            .withColumn("japanese_content", F.when(df.content_language == "ja", df.content))
            .withColumn("japanese_reply", F.when(df.reply_language == "ja", df.reply))
            .withColumn("japanese_title", F.when(df.title_language == "ja", df.title))
            .withColumn("korean_content", F.when(df.content_language == "ko", df.content))
            .withColumn("korean_reply", F.when(df.reply_language == "ko", df.reply))
            .withColumn("korean_title", F.when(df.title_language == "ko", df.title))
            .withColumn("other_language_content", F.when(
            (df.content_language != "zh") & (df.content_language != "zh_Hant") & (df.content_language != "en") &
            (df.content_language != "ja") & (df.content_language != "ko"), df.content))
            .withColumn("other_language_reply", F.when(
            (df.reply_language != "zh") & (df.reply_language != "zh_Hant") & (df.reply_language != "en") &
            (df.reply_language != "ja") & (df.reply_language != "ko"), df.reply))
            .withColumn("other_language_title", F.when(
            (df.title_language != "zh") & (df.title_language != "zh_Hant") & (df.title_language != "en") &
            (df.title_language != "ja") & (df.title_language != "ko"), df.title))
            .collect()
    )
    samples.append((idx, rows))
print samples


In [0]:
% pyspark
import json
import time
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get


def compare(doc, row):
    keys = [u"review_id", u'_identifier', u'app_id', u'chinese_content', u'chinese_reply', u'chinese_title', u'content',
            u'content_language', u'country_code', u'date', u'device_code', u'english_content', u'english_reply',
            u'english_title', u'japanese_content', u'japanese_reply', u'japanese_title', u'korean_content',
            u'korean_reply', u'korean_title', u'language', u'market_code', u'other_language_content',
            u'other_language_reply', u'other_language_title', u'product_version', u'rating', u'reply', u'reply_date',
            u'reply_language', u'title', u'title_language', u'user_device', u'user_id', u'user_language', u'user_name',
            u'user_purchased']
    row_dict = row.asDict()
    for key in keys:
        if key in ("app_id", "_identifier"):
            assert str(doc[key]) == str(row_dict[key]), "{} is diffrent".format(key)
        else:
            assert doc[key] == row_dict[key], "{} is diffrent".format(key)


def es_doc(index, doc_id, routing_id):
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth("***", "***")

    url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/{}/_doc/{}?routing={}".format(
        index, doc_id, routing_id)
    resp = plain_get(url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
    doc = json.loads(resp.text)
    source = doc['_source']
    source['review_id'] = doc["_id"]
    return source


for idx, rows in samples:
    for row in rows:
        doc = es_doc(idx, row.review_id, row.app_id)
        compare(doc, row)


In [0]:

# review_df = spark.read.parquet(
#     "s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/event_month=2018-01")
# review_df.registerTempTable("reviews")
ios_samples_df = spark.sql("SELECT review_id,time, process_date, process_hour FROM reviews WHERE market_code = 'google-play' group by review_id , time , process_date, process_hour  LIMIT 10")
print ios_samples_df.show()


In [0]:

from pyspark.sql.functions import desc
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review_load_temp/review_load_temp_v4/", depth_is_1=True)
print path_list[107]
for path in path_list[107:]:
    dup_list = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/{}/'.format(path.split("/")[3])).select('review_id','process_date','process_hour').distinct().groupBy("review_id").agg({"*":"count"}).where("count(1)>1").orderBy("count(1)",ascending = False).take(3)

    for x in dup_list:
        result1 = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/{}'.format(path.split("/")[3])).filter("review_id='{}'".format(x["review_id"])).groupBy("process_date","process_hour","time").agg({"*":"count"}).orderBy(desc("time"),desc("process_date"),desc("process_hour")).head(1)
        result2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v1/{}".format(path.split("/")[3])).filter("review_id='{}'".format(x["review_id"])).head(1)
        # print result1,result2
        if result1[0]["process_date"]==result2[0]["process_date"] and result1[0]["process_hour"]==result2[0]["process_hour"] and result1[0]["time"]==result2[0]["time"]  :
            print 'pass', path
        else:
            print 'failed!!!',result1, result2,x

In [0]:

from pyspark.sql.functions import desc

dup_list = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/event_month=2019-09/').select('review_id','process_date','process_hour').distinct().groupBy("review_id").agg({"*":"count"}).where("count(1)>1").orderBy("count(1)",ascending = False).take(10)

for x in dup_list:
    result1 = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/event_month=2019-09').filter("review_id='{}'".format(x["review_id"])).groupBy("process_date","process_hour","time").agg({"*":"count"}).orderBy(desc("time"),desc("process_date"),desc("process_hour")).take(1)
    result2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v1/event_month=2019-09").filter("review_id='{}'".format(x["review_id"])).take(1)
    print result1,result2
    if result1[0]["process_date"]==result2[0]["process_date"] and result1[0]["process_hour"]==result2[0]["process_hour"] and result1[0]["time"]==result2[0]["time"]  :
        print 'pass'
    else:
        print 'failed!!!',result1, result2,x

In [0]:

from aadatapipelinecore.core.log import logger

import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review_load_temp/review_load_temp_v4/", depth_is_1=True)
for path in path_list[107:]:
    count_1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/{}".format(path)).select("review_id").distinct().count()
    count_2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/{}".format(path.split("/")[3])).select("review_id").count()
    if count_1 != count_2:
        print "not equal!!!!!!", count_1 , count_2, path
    else:
        print 'pass', path

In [0]:

from aadatapipelinecore.core.fs.driver import unified
from aadatapipelinecore.core.utils.identifier import virtual_id
from aadatapipelinecore.core.urn import Urn


def urn(namespace):
    from aadatapipelinecore.core.urn import Urn
    from aadatapipelinecore.core.pipeline import type_
    urn = Urn(
        identifier=virtual_id(),
        event=type_.EventType.TRANSFORM,
        manipulation=type_.ManipulationType.QUERY,
        namespace=namespace,
        data_type="fact"
    )
    return urn
test_urn = urn("int-ss.review.v1")

unified.read(spark, test_urn).urn
# print unified.read(spark, test_urn,sql_where="process_granularity='monthly' and process_date=2017-01-31'").data_dto_list

# take(1)[0]


In [0]:
%%sh
curl -u bdp_qa:6trmJcZBJrKWdtN2 -Get http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/int-ss-review_v2-apple-store-ios-all-201703/_search?pretty

In [0]:

from pyspark.sql.functions import countDistinct
# spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/MONTHLY/2017-01-31/gp/",sep="\t").filter("_c0='gp:AOqpTOHUKdAGUV_C94UVZHgqaXl9iqnCCdE-8Uqp3vdNW89Qt6UV9ADIOCo9P73sEEXBRFnDmALiiklCpHj9Gr4'").show(20,False,vertical=False)

print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/").filter("market_code='google-play' and _identifier='220200118161416198'").cache().agg(countDistinct("review_id")).collect()
unified:30102239 
db:30191242

In [0]:
%%sh%pyspark
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pyspark.storagelevel import StorageLevel
from pyspark.sql import functions as F
start_month = datetime.strptime("2017-06","%Y-%m")
end_month = datetime.strptime("2020-01","%Y-%m")
a_month = relativedelta(months=1)
now = start_month
while now <= end_month:
    path = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity={weekly,daily,hourly}/process_date=%s-*/" % now.strftime("%Y-%m")
    print "%s started" % path 
    review_df = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/").parquet(path).where("process_granularity in ('daily', 'weekly', 'hourly') and market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all')")
    review_df = review_df.withColumn("event_month",F.from_unixtime(F.unix_timestamp("time", "yyyy-MM-dd"), "yyyy-MM")).withColumn("event_date",F.from_unixtime(F.unix_timestamp("time", "yyyy-MM-dd"), "yyyy-MM-dd")).repartition(1000, "event_date").drop("event_date")
    review_df.write.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v2/", mode="append", partitionBy=["event_month"])
    now += a_month
    print "%s ended" % path

